In [33]:
from dask.distributed import Client
from dask_kubernetes.operator import KubeCluster
import subprocess

In [32]:
subprocess.run("gcloud container clusters resize coolercluster --node-pool workerpool --num-nodes 0 --zone us-central1-a --quiet" , shell=True)

Resizing coolercluster...
................................done.
Updated [https://container.googleapis.com/v1/projects/extreme-lore-398917/zones/us-central1-a/clusters/coolercluster].


CompletedProcess(args='gcloud container clusters resize coolercluster --node-pool workerpool --num-nodes 0 --zone us-central1-a --quiet', returncode=0)

In [34]:
class MyClient(Client):
    
    def __exit__(self, *args, **kwargs):
        super().__exit__(*args, **kwargs)
        subprocess.run("gcloud container clusters resize coolercluster --node-pool workerpool --num-nodes 0 --zone us-central1-a --quiet" , shell=True)        

In [37]:
cluster: KubeCluster = KubeCluster(name="mycluster",
                                   image='hermelinkluntjes/thesis:test',
                                   n_workers=0)

Output()

In [ ]:
client = MyClient(cluster)

/home/beer/miniconda3/envs/test/lib/python3.10/site-packages/distributed/client.py:1391: VersionMismatchWarning: Mismatched versions found

+-------------+--------+-----------+---------+
| Package     | Client | Scheduler | Workers |
+-------------+--------+-----------+---------+
| cloudpickle | 3.0.0  | 2.2.1     | None    |
| lz4         | None   | 4.3.2     | None    |
| msgpack     | 1.0.7  | 1.0.6     | None    |
| tornado     | 6.1    | 6.3.3     | None    |
+-------------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [25]:
# client.wait_for_workers(2)
client

Connection method: Cluster object,Cluster type: __main__.MyKubeCluster
Dashboard: http://localhost:54256/status,
Dashboard: http://localhost:54256/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.0.17:8786,Workers: 0
Dashboard: http://10.12.0.17:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [12]:
worker_group_config = {
   "name": "highmem",
   "n_workers": 2,
   "resources":{
      "requests": 
         {"memory": "6Gi",
          "cpu": "3"}
      }
}
cluster.add_worker_group(**worker_group_config)

In [26]:
cluster.scale(2, worker_group="highmem")

Task exception was never retrieved
future: <Task finished name='Task-4997' coro=<PortForward._sync_sockets() done, defined at /home/beer/miniconda3/envs/test/lib/python3.10/site-packages/kr8s/_portforward.py:167> exception=ExceptionGroup('unhandled errors in a TaskGroup', [ConnectionClosedError('TCP socket closed')])>
  + Exception Group Traceback (most recent call last):
  |   File "/home/beer/miniconda3/envs/test/lib/python3.10/site-packages/kr8s/_portforward.py", line 171, in _sync_sockets
  |     async with anyio.create_task_group() as tg:
  |   File "/home/beer/miniconda3/envs/test/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 664, in __aexit__
  |     raise BaseExceptionGroup(
  | exceptiongroup.ExceptionGroup: unhandled errors in a TaskGroup (1 sub-exception)
  +-+---------------- 1 ----------------
    | Traceback (most recent call last):
    |   File "/home/beer/miniconda3/envs/test/lib/python3.10/site-packages/kr8s/_portforward.py", line 183, in _tcp_to_ws
 

NotFoundError: Object mycluster-highmem does not exist

In [ ]:
cluster.scale(0, worker_group="highmem")

Task exception was never retrieved
future: <Task finished name='Task-4815' coro=<PortForward._sync_sockets() done, defined at /home/beer/miniconda3/envs/test/lib/python3.10/site-packages/kr8s/_portforward.py:167> exception=HTTPStatusError("Client error '401 Unauthorized' for url 'https://34.30.34.80/api/v1/namespaces/default/pods?labelSelector=dask.org%2Fcluster-name%3Dmycluster%2Cdask.org%2Fcomponent%3Dscheduler'\nFor more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/401")>
Traceback (most recent call last):
  File "/home/beer/miniconda3/envs/test/lib/python3.10/site-packages/kr8s/_portforward.py", line 170, in _sync_sockets
    async with self._connect_websocket() as ws:
  File "/home/beer/miniconda3/envs/test/lib/python3.10/contextlib.py", line 199, in __aenter__
    return await anext(self.gen)
  File "/home/beer/miniconda3/envs/test/lib/python3.10/site-packages/kr8s/_portforward.py", line 147, in _connect_websocket
    self.pod = await self._select_p

In [17]:
cluster.delete_worker_group("highmem")

Task exception was never retrieved
future: <Task finished name='Task-4829' coro=<PortForward._sync_sockets() done, defined at /home/beer/miniconda3/envs/test/lib/python3.10/site-packages/kr8s/_portforward.py:167> exception=HTTPStatusError("Client error '401 Unauthorized' for url 'https://34.30.34.80/api/v1/namespaces/default/pods?labelSelector=dask.org%2Fcluster-name%3Dmycluster%2Cdask.org%2Fcomponent%3Dscheduler'\nFor more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/401")>
Traceback (most recent call last):
  File "/home/beer/miniconda3/envs/test/lib/python3.10/site-packages/kr8s/_portforward.py", line 170, in _sync_sockets
    async with self._connect_websocket() as ws:
  File "/home/beer/miniconda3/envs/test/lib/python3.10/contextlib.py", line 199, in __aenter__
    return await anext(self.gen)
  File "/home/beer/miniconda3/envs/test/lib/python3.10/site-packages/kr8s/_portforward.py", line 147, in _connect_websocket
    self.pod = await self._select_p

In [27]:
with cluster as  c:
    print(c)

MyKubeCluster(mycluster, 'tcp://localhost:55279', workers=0, threads=0, memory=0 B)


RuntimeError: super(): no arguments